# <div style="color:white;display:fill;border-radius:5px;background-color:#0E2031;letter-spacing:0.5px;overflow:hidden"><p style="padding:20px;color:white;overflow:hidden;text-align: center;margin:0;font-size:120%">Content</p></div>

**Collecting Train Data from AWS**  
- Loading AWS Credencials
- Checking Schemas, Databases, Tables

**Collecting Test Data from Kaggle**
- Loading Kaggle Credencials

In [1]:
#import sqlite3
from sqlalchemy               import create_engine
from sqlalchemy.pool          import NullPool

import os
import sys
import psycopg2
import dotenv
import zipfile
import kaggle
from kaggle.api.kaggle_api_extended import KaggleApi

import inflection
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
print(sys.executable)

C:\Users\Alysson\anaconda3\python.exe


In [4]:
#pip install python-dotenv
#pip install kaggle

# <div style="color:white;display:fill;border-radius:5px;background-color:#0E2031;letter-spacing:0.5px;overflow:hidden"><p style="padding:20px;color:white;overflow:hidden;text-align: center;margin:0;font-size:120%">Collecting Train Data from AWS</p></div>

## <div style="color:white;display:fill;border-radius:5px;background-color:#153656;letter-spacing:0.5px;overflow:hidden"><p style="padding:15px;color:white;overflow:hidden;text-align: center;margin:0;font-size:120%">Loading Credentials</p></div>

In [5]:
# Loading local credencials 
dotenv.load_dotenv(dotenv.find_dotenv())

True

In [6]:
host = os.environ.get('POSTGRES_HOST')
port = os.environ.get('POSTGRES_PORT')
database = os.environ.get('POSTGRES_DATABASE')
user = os.environ.get('POSTGRES_USER')
pwd = os.environ.get('POSTGRES_PWD')

In [7]:
#host = '********.*******.us-east-1.rds.amazonaws.com'
#port = ****
#database = '************'
#user = '*******'
#pwd = '*******'

In [8]:
endpoint= f'postgresql://{user}:{pwd}@{host}:{port}/{database}'
        
db = create_engine(endpoint, poolclass=NullPool)
#db = create_engine(endpoint, pool_size=50, echo=False)
conn = db.connect()

In [11]:
#Select distinct schemas
query = "SELECT distinct(table_schema) from information_schema.tables"
df = pd.read_sql( query, db )
df.head()

,table_schema
0,public
1,pg_catalog
2,information_schema
3,pa004


In [12]:
#Select tables from schema
query = "SELECT * FROM information_schema.tables WHERE table_schema = 'pa004'"
df = pd.read_sql( query, db )
df.head()

,table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action
0,comunidadedsdb,pa004,users,BASE TABLE,None,None,None,None,None,YES,NO,None
1,comunidadedsdb,pa004,vehicle,BASE TABLE,None,None,None,None,None,YES,NO,None
2,comunidadedsdb,pa004,insurance,BASE TABLE,None,None,None,None,None,YES,NO,None


In [13]:
#Consult database pa004 table insurance
query = """ SELECT * FROM pa004.insurance """
insurance_raw = pd.read_sql_query(query, conn)
insurance_raw.head()

,id,previously_insured,annual_premium,vintage,response
0,1,0,40454.0,217,1
1,2,0,33536.0,183,0
2,3,0,38294.0,27,1
3,4,1,28619.0,203,0
4,5,1,27496.0,39,0


In [14]:
#consult database pa004 table users
query = """ SELECT * FROM pa004.users """
users_raw = pd.read_sql_query(query, conn)
users_raw.head()

,id,gender,age,region_code,policy_sales_channel
0,1,Male,44,28.0,26.0
1,2,Male,76,3.0,26.0
2,3,Male,47,28.0,26.0
3,4,Male,21,11.0,152.0
4,5,Female,29,41.0,152.0


In [15]:
#consult database pa004 table vehicle
query = """ SELECT * FROM pa004.vehicle """
vehicle_raw = pd.read_sql_query(query, conn)
vehicle_raw.head()

,id,driving_license,vehicle_age,vehicle_damage
0,1,1,> 2 Years,Yes
1,2,1,1-2 Year,No
2,3,1,> 2 Years,Yes
3,4,1,< 1 Year,No
4,5,1,< 1 Year,No


In [16]:
#build raw dataset:
query = """
SELECT *
FROM pa004.users u 
INNER JOIN pa004.vehicle v on u.id = v.id 
INNER JOIN pa004.insurance i ON u.id = i.id
ORDER BY u.id
"""
df_raw = pd.read_sql( query, db )
df_raw.head()

,id,gender,age,region_code,policy_sales_channel,id,driving_license,vehicle_age,vehicle_damage,id,previously_insured,annual_premium,vintage,response
0,1,Male,44,28.0,26.0,1,1,> 2 Years,Yes,1,0,40454.0,217,1
1,2,Male,76,3.0,26.0,2,1,1-2 Year,No,2,0,33536.0,183,0
2,3,Male,47,28.0,26.0,3,1,> 2 Years,Yes,3,0,38294.0,27,1
3,4,Male,21,11.0,152.0,4,1,< 1 Year,No,4,1,28619.0,203,0
4,5,Female,29,41.0,152.0,5,1,< 1 Year,No,5,1,27496.0,39,0


In [ ]:
conn.close()

In [17]:
df_raw = df_raw.loc[:,~df_raw.columns.duplicated()]

In [19]:
datapath = '../data/'
df_raw.to_csv(datapath+'train_raw.csv')

## <div style="color:white;display:fill;border-radius:5px;background-color:#153656;letter-spacing:0.5px;overflow:hidden"><p style="padding:15px;color:white;overflow:hidden;text-align: center;margin:0;font-size:120%">Collecting Test Data from Kaggle</p></div>

In [26]:
#!pip install kaggle

In [27]:
#!mkdir ~/.kaggle 

In [79]:
api = KaggleApi()
api.authenticate()

In [80]:
#!kaggle datasets list -s 'health-insurance-cross-sell-prediction'

In [111]:
api.dataset_download_file('anmolkumar/health-insurance-cross-sell-prediction', file_name='test.csv')

True

In [112]:
with zipfile.ZipFile('test.csv.zip','r') as zipref:
    zipref.extractall()

In [113]:
df_raw = pd.read_csv('test.csv')
df_raw.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage
0,381110,Male,25,1,11.0,1,< 1 Year,No,35786.0,152.0,53
1,381111,Male,40,1,28.0,0,1-2 Year,Yes,33762.0,7.0,111
2,381112,Male,47,1,28.0,0,1-2 Year,Yes,40050.0,124.0,199
3,381113,Male,24,1,27.0,1,< 1 Year,Yes,37356.0,152.0,187
4,381114,Male,27,1,28.0,1,< 1 Year,No,59097.0,152.0,297


In [102]:
# snakecase = lambda x: inflection.underscore(x)
# cols_new = list(map( snakecase, df_raw.columns))
# df_raw.columns = cols_new

In [116]:
lowecase = lambda x: str.lower(x)
cols_new = list(map( lowecase, df_raw.columns))
df_raw.columns = cols_new

In [118]:
df_raw

,id,gender,age,driving_license,region_code,previously_insured,vehicle_age,vehicle_damage,annual_premium,policy_sales_channel,vintage
0,381110,Male,25,1,11.0,1,< 1 Year,No,35786.0,152.0,53
1,381111,Male,40,1,28.0,0,1-2 Year,Yes,33762.0,7.0,111
2,381112,Male,47,1,28.0,0,1-2 Year,Yes,40050.0,124.0,199
3,381113,Male,24,1,27.0,1,< 1 Year,Yes,37356.0,152.0,187
4,381114,Male,27,1,28.0,1,< 1 Year,No,59097.0,152.0,297
...,...,...,...,...,...,...,...,...,...,...,...
127032,508142,Female,26,1,37.0,1,< 1 Year,No,30867.0,152.0,56
127033,508143,Female,38,1,28.0,0,1-2 Year,Yes,28700.0,122.0,165
127034,508144,Male,21,1,46.0,1,< 1 Year,No,29802.0,152.0,74
127035,508145,Male,71,1,28.0,1,1-2 Year,No,62875.0,26.0,265


In [119]:
df_raw = df_raw.to_csv('test_raw.csv')

In [106]:
#!mv notebooks/test.csv data/test_raw.csv